# Урок 3: Post-Training Weight-Activation Quantization (W8A8)

Квантование и весов, и активаций (выходов слоев). Это необходимо для реального ускорения вычислений на Tensor Cores в режиме INT8/INT4.

## 1. Математическая теория

### 1.1. Проблема выбросов в активациях
В LLM активации имеют «тяжелые хвосты» — небольшое количество каналов имеет огромные значения. Это делает стандартное квантование активаций невозможным без огромных потерь точности.

### 1.2. Методы из обзора:
*   **SmoothQuant (Xiao et al., 2023):** Математически переносит сложность квантования с активаций на веса. 
    $$Y = (X \cdot diag(s)^{-1}) \cdot (diag(s) \cdot W)$$
    Мы подбираем вектор $s$ так, чтобы подавить выбросы в $X$, ценой увеличения размаха весов $W$ (которые квантовать гораздо проще).
*   **LLM.int8() (Dettmers et al., 2022):** Адаптивный метод. Выделяет «выбросные» каналы и считает их отдельно в FP16, а остальную массу (99.9%) — в INT8.
*   **RPTQ (Yuan et al., 2023a):** Группирует каналы активаций по их диапазонам (Reordering), применяя разные параметры квантования к разным группам.
*   **OmniQuant (Shao et al., 2024b):** Использует обучаемые параметры (learnable scales and clipping) для оптимизации процесса квантования без полноценного fine-tuning.

---

In [ ]:
import torch
from src.model import GPTLanguageModel, device

def smoothquant_logic_raw(W, X, alpha=0.5):
    # Вычисляем scale s
    act_max = X.abs().max(dim=0)[0]
    weight_max = W.abs().max(dim=0)[0]
    s = act_max.pow(alpha) / weight_max.pow(1-alpha)
    return s

print("nanoGPT Track: Реализована математика SmoothQuant (расчет вектора s).")

## 2. Промышленная реализация: BitsAndBytes
Библиотека `bitsandbytes` позволяет загружать любую модель `transformers` в режиме 8-бит (W8A8) одной командой.

In [ ]:
from transformers import AutoModelForCausalLM
try:
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model_8bit = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")
    print("Llama Track: Модель загружена в режиме LLM.int8() (W8A8).")
except Exception as e:
    print(f"Ошибка: {e}")